In [16]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import dateutil.parser
import json
from pylab import rcParams
import pickle
import category_encoders as ce
import ipaddress
import glob
import os
import joblib

def get_ordinal_mapping_kondo(obj):#救世主近藤さん#カテゴリカル化した際の対応表
    categorical_values = {}
    for x in obj.category_mapping:
        categorical_values[x['col']] = inverse_dict(x['mapping'].to_dict())
    map_df = pd.DataFrame(categorical_values)
    #map_df = pd.DataFrame(categorical_values,columns=['column', 'label', 'ord_num'])
    #print(categorical_values)
    return map_df 

def drop_numbers(df):#カテゴリカル化に関係ない数字や時刻などの列を削除，あとで結合する
    #中身数字削除
    temp = df.select_dtypes(exclude='number')
    df_others = df.select_dtypes(include='number')
    #中身時刻削除
    df_target = temp.drop('time',axis=1)#列削除はaxis=1
    df_others['time'] = df['time']
    #labelの場合はmappingが入れ子構造になっているので削除
    if 'message' in df_target.columns:
        #print('this is it')
        df_others['message'] = df_target['message']
        df_others['recipes'] = df_target['recipes']
        df_target_new = df_target.drop(['message','recipes'],axis=1)
        if 'message' in df_target_new.columns:
            print('fuck')
        return df_target_new, df_others
    return df_target, df_others

def inverse_dict(d):
    return {v:k for k,v in d.items()}

def drop_cols(df):#dfを受け取るとカテゴリカル化して数字や時刻列なども再度結合したdataframeと対応表となっているdataframeを返す
    df_target,df_others = drop_numbers(df)
    cols_list = df_target.columns
    OrdinalEncoder = ce.OrdinalEncoder(cols=cols_list,handle_unknown='ignore')
    df_OE = OrdinalEncoder.fit_transform(df_target)
    df_OE_merged = pd.concat([df_OE,df_others],axis=1)
    #df_correspondence = get_ordinal_mapping2(OrdinalEncoder)
    df_correspondence = get_ordinal_mapping_kondo(OrdinalEncoder)
    return df_OE_merged, df_correspondence

In [23]:
#ディレクトリ内のpickleからdataframeを作成
#まつけんが作ってくれたdropped_dfのdataframeを処理
#近藤さんがdropped_df2を作成してくださったのでそれにあわせて変更
network_device='/data/output-directory_all/df/network-device-bgpnw2/dropped_df2'
physical_infra='/data/output-directory_all/df/physical-infrastructure-bgpnw2/dropped_df2'
#virtual_infra_devices='/data/output-directory_all/df/virtual-infrastructure-bgpnw2/devices/dropped_df'
virtual_infra_devices='/data/output-directory_all/df/virtual-infrastructure-bgpnw2/devices/dropped_df2'
virtual_infra_networks='/data/output-directory_all/df/virtual-infrastructure-bgpnw2/networks/dropped_df2'
virtual_infra_ports='/data/output-directory_all/df/virtual-infrastructure-bgpnw2/ports/dropped_df2'
label_all='/data/output-directory_all/df/label'
label_random='/data/output-directory_random/df/label'

####ここを変更することでどのdataframe群(たとえばnetwork device)を扱うか決定する
DATA_DIR = label_random
####新しく作ったdfの保存ディレクトリは選択したディレクトリの下
NEW_DIR = '/categorical/'#保存先ディレクトリ
os.makedirs(DATA_DIR+NEW_DIR,exist_ok=True)



#指定したパスのディレクトリ内のすべてのpickleを取得する
files = glob.glob(os.path.join(DATA_DIR,'*.pkl'))
lists_df=[]
df={}
df_new={}
df_map={}
#print(files)
i=0
for file in files:#ひとつずつpickleを処理し下のディレクトリに保存
    #print(file)
    filename = os.path.splitext(os.path.basename(file))[0]
    #print(filename)
    #df[filename] = joblib.load(file)
    df[filename] = pd.read_pickle(file)
    lists_df.append(filename)#どのようなdataframeを取得したか記録
    #ordinal encoding処理をして新しいディレクトリにpickleとして保存
    df_new[filename],df_map[filename] = drop_cols(df[filename])
    
    df_new[filename].to_pickle(DATA_DIR+NEW_DIR+"{0}.pkl".format(filename))
    df_map[filename].to_pickle(DATA_DIR+NEW_DIR+"map_{0}.pkl".format(filename))
    
#print(len(lists_df))

/home/t_ubukata/.local/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/t_ubukata/.local/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/t_ubukata/.local/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

/home/t_ubukata/.local/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/t_ubukata/.local/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/t_ubukata/.local/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https: